<a href="https://colab.research.google.com/github/yyyhpets2022/CSI5386-Natural-Language-Processing/blob/main/CSI_5386_NPL_Assignment_2_Part_2_%7C_S_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Similarity with BERT

**Credit to:** Mohamad Merchant <br>
**Reference link:** https://keras.io/examples/nlp/semantic_similarity_with_bert/ <br>
**GitHub of initial code:** https://github.com/keras-team/keras-io <br>
**Description:** Natural Language Inference by fine-tuning BERT model on SNLI Corpus.

## Introduction

Semantic Similarity is the task of determining how similar
two sentences are, in terms of what they mean.
This example demonstrates the use of SNLI (Stanford Natural Language Inference) Corpus
to predict sentence semantic similarity with Transformers.
We will fine-tune a BERT model that takes two sentences as inputs
and that outputs a similarity score for these two sentences.

### References

* [BERT](https://arxiv.org/pdf/1810.04805.pdf)
* [SNLI](https://nlp.stanford.edu/projects/snli/)

## Setup

Note: install HuggingFace `transformers` via `pip install transformers` (version >= 2.11.0).

In [ ]:
pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 22.7 MB/s 
     |████████████████████████████████| 7.6 MB 52.6 MB/s 
     |████████████████████████████████| 182 kB 63.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

## Updating data set

Please make sure to have the proper file name in the pd.read_excel function

In [ ]:
assignment_df = pd.read_excel("/content/STS2016.input.question-question.txt - similarity score.xlsx")

assignment_df["Similarity labels"] = pd.cut(x=assignment_df["Score S-BERT times 5"], 
                             bins=[-2.5, 0, 2.5, 5], 
                             labels=["contrary", "neutral", "similar"])

del assignment_df['Unnamed: 5']
del assignment_df['Unnamed: 6']
del assignment_df['Unnamed: 7']

assignment_df

,Sentence 1,Sentence 2,Score S-BERT,Score S-BERT times 5,Assignment Score,Similarity labels
0,Should I drink water during my workout?,How can I get my toddler to drink more water?,0.2342,1.1710,NaN,neutral
1,How can I put something in book format without...,"How can I ""time-stamp"" my data without publish...",0.3888,1.9440,NaN,neutral
2,How do I stop my dog from jumping on me?,How do I make my dog forget a command?,0.4675,2.3375,NaN,neutral
3,What's the best way to store asparagus?,What's the correct way to store fats?,0.4558,2.2790,NaN,neutral
4,How do I make a height adjustable desk?,How can I build a wall mounted adjustable heig...,0.8776,4.3880,4.0,similar
...,...,...,...,...,...,...
1550,what is the difference between Erebor and Moria?,What is the difference between sortition and d...,0.2578,1.2890,NaN,neutral
1551,What window part do I need for this window?,Do I need a header for an egress window?,0.3502,1.7510,NaN,neutral
1552,How to test if a toilet is leaking?,How can I test if a dehydrator is getting to t...,0.4073,2.0365,NaN,neutral
1553,How do I test the functionality of a sump pump?,Do I really need a redundant sump pump?,0.4946,2.4730,NaN,neutral


In [ ]:
# Drop missing value rows:
initial_df = assignment_df.dropna()
initial_df.reset_index()

,index,Sentence 1,Sentence 2,Score S-BERT,Score S-BERT times 5,Assignment Score,Similarity labels
0,4,How do I make a height adjustable desk?,How can I build a wall mounted adjustable heig...,0.8776,4.3880,4.0,similar
1,6,How can I find out why my washing machine trip...,How can I figure out why my washing machine is...,0.8225,4.1125,4.0,similar
2,8,Should I use IRA money to pay down my student ...,Should I cash out my IRA to pay my student loans?,0.8918,4.4590,5.0,similar
3,14,What is the best way to repair a cracked bathtub?,What is the best way to fix this garage floor?,0.2906,1.4530,0.0,neutral
4,20,What's the best way to store asparagus?,What's the best way to store unused sushi rice?,0.5935,2.9675,0.0,similar
...,...,...,...,...,...,...,...
204,1523,How to make good coffee in a Moka pot?,How to make more than one good cup of coffee w...,0.8548,4.2740,4.0,similar
205,1524,How do I prepare this porous interior wall for...,How do I install a new interior partition wall?,0.3837,1.9185,1.0,neutral
206,1527,What could be causing my GFCI to trip?,What could be causing my GFCI outlet to trip?,0.9038,4.5190,4.0,similar
207,1536,How do I prepare this porous interior wall for...,How do I make this paint match?,0.4186,2.0930,1.0,neutral


## Configuration

In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = ["contrary", "neutral", "similar"]

## Load the Data

In [ ]:
# There are more than 550k samples in total; we will use 100k for this example.
train_df = initial_df
valid_df = initial_df
test_df = initial_df

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")

Total train samples : 209
Total validation samples: 209
Total test samples: 209


Dataset Overview:

- sentence1: The premise caption that was supplied to the author of the pair.
- sentence2: The hypothesis caption that was written by the author of the pair.
- similarity: This is the label chosen by the majority of annotators.
Where no majority exists, the label "-" is used (we will skip such samples here).

Here are the "similarity" label values in our dataset:

- Contrary: The sentences share no similarity.
- Similar: The sentences have similar meaning.
- Neutral: The sentences are neutral.

Let's look at one sample from the dataset:

## Preprocessing

In [ ]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
Sentence 1              0
Sentence 2              0
Score S-BERT            0
Score S-BERT times 5    0
Assignment Score        0
Similarity labels       0
dtype: int64


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


One-hot encode training, validation, and test labels.

In [ ]:
train_df["label"] = train_df["Similarity labels"].apply(
    lambda x: 0 if x == "contrary" else 1 if x == "similar" else 2
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["Similarity labels"].apply(
    lambda x: 0 if x == "contrary" else 1 if x == "similar" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["Similarity labels"].apply(
    lambda x: 0 if x == "contrary" else 1 if x == "similar" else 2
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [ ]:
train_df["label"]

4       1
6       1
8       1
14      2
20      1
       ..
1523    1
1524    2
1527    1
1536    2
1549    1
Name: label, Length: 209, dtype: category
Categories (3, int64): [0 < 2 < 1]

## Keras Custom Data Generator

In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


## Build the model.

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f3b0a819a90>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',        

Create train and validation data generators

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["Sentence 1", "Sentence 2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["Sentence 1", "Sentence 2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

## Train the Model

Training is done only for the top layers to perform "feature extraction",
which will allow the model to use the representations of the pretrained model.

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
6/6 [==============================] - ETA: 0s - loss: 0.7473 - acc: 0.6250 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


6/6 [==============================] - 187s 30s/step - loss: 0.7473 - acc: 0.6250 - val_loss: 0.5991 - val_acc: 0.6771
Epoch 2/2
6/6 [==============================] - 160s 29s/step - loss: 0.6353 - acc: 0.6667 - val_loss: 0.5463 - val_acc: 0.8125


## Fine-tuning

This step must only be performed after the feature extraction model has
been trained to convergence on the new data.

This is an optional last step where `bert_model` is unfreezed and retrained
with a very low learning rate. This can deliver meaningful improvement by
incrementally adapting the pretrained features to the new data.

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

# Train the entire model end-to-end.

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2


6/6 [==============================] - 354s 57s/step - loss: 0.5427 - accuracy: 0.7344 - val_loss: 0.4925 - val_accuracy: 0.7917
Epoch 2/2
6/6 [==============================] - 320s 56s/step - loss: 0.4902 - accuracy: 0.7865 - val_loss: 0.4457 - val_accuracy: 0.8385


## Evaluate model on the test set

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["Sentence 1", "Sentence 2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


6/6 [==============================] - 72s 12s/step - loss: 0.4457 - accuracy: 0.8385


[0.4457043707370758, 0.8385416865348816]

## Inference on custom sentences

In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = 5*model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = float("{:.4f}".format(proba[idx]))
    pred = labels[idx]
    return proba


Check results on some example sentence pairs.

In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 6s 6s/step


3.4627

Check results on some example sentence pairs.

In [ ]:
sentence1 = "A smiling costumed woman is holding an umbrella"
sentence2 = "A happy woman in a fairy costume holds an umbrella"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 621ms/step


3.8482

Check results on some example sentence pairs

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 572ms/step


3.5973

Checking the scores for the first two sentences of the initial dataframe

In [ ]:
list1 = []
score = check_similarity(assignment_df["Sentence 1"][0], assignment_df["Sentence 2"][0])
score1 = check_similarity(assignment_df["Sentence 1"][1], assignment_df["Sentence 2"][1])
list1.append(score)
list1.append(score1)
print(list1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 617ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 572ms/step
[3.8196, 3.979]


Applying the similarity scores on the entire initial dataframe

In [ ]:
new_score = []
for i in range(len(assignment_df)):
  score = check_similarity(assignment_df["Sentence 1"][i], assignment_df["Sentence 2"][i])
  new_score.append(score)

print(new_score)

Adding the new scores to the dataframe

In [ ]:
# Adding the new scores to the assignment_df 
assignment_df.insert(loc=6,
              column="New Similarity Score",
              value=new_score)

del assignment_df['Similarity labels']

Saving the dataframe as a CSV file

In [ ]:
# Saving the dataframe with the new scores as a CSV 
csv_name_based_V2 = r'New.STS2016.input.question-question.csv'
assignment_df.to_csv(csv_name_based_V2, index = False)